In [1]:
import pyiron
import numpy as np
import matplotlib.pylab as plt
import ase.units

In [2]:
pr = pyiron.Project('tensile test')

In [3]:
structure = pr.create.structure.bulk('Fe', cubic=True)

In [4]:
gb = pr.create.structure.aimsgb.build(axis=(1,0,0), sigma=5, plane=(1,0,0), initial_struct=structure,uc_a=2, uc_b=2)
gb.plot3d()

/home/gu.huang/miniconda3/envs/pyiron/lib/python3.12/site-packages/structuretoolkit/build/aimsgb.py:97: UserWarning: `add_if_dist` is deprecated, please use `gap` instead.
  warn("`add_if_dist` is deprecated, please use `gap` instead.")


NGLWidget()

In [5]:
len(gb)

80

In [6]:
gb = pr.create.structure.aimsgb.build(axis=(1,-1,0), sigma=9, plane=(1,-1,0), initial_struct=structure,uc_a=12, uc_b=12)
gb.plot3d()

NGLWidget()

In [7]:
gb.get_cell()

Cell([[8.61, 0.0, 5.272104470329355e-16], [1.9581110494347007e-15, 12.176378772032347, 7.455881644187597e-16], [0.0, 0.0, 97.41103017625878]])

In [8]:
gb = gb.repeat([6,4,1])

In [9]:
len(gb)

20736

In [10]:
gb.get_cell()

Cell([[51.66, 0.0, 3.163262682197613e-15], [7.832444197738803e-15, 48.70551508812939, 2.9823526576750387e-15], [0.0, 0.0, 97.41103017625878]])

In [12]:
# optimise
job = pr.create.job.Lammps('minimize', delete_existing_job=True)
job.structure = gb
job.potential = 'FeH-NNIP'
job.calc_minimize(pressure=[None, None, 0.0, None, None, None])
job.server.cores = 16
job.run(delete_existing_job=True)

The job minimize was saved and received the ID: 10941


In [13]:
job.animate_structures()

NGLWidget(max_frame=38)

In [14]:
structure = job.get_structure(frame=-1)
structure.get_cell()

Cell([[51.66, -5.144298926031891e-15, 3.163262682197613e-15], [-1.8261643180703176e-31, 48.705515088, 2.9823526576671158e-15], [-5.838245629166617e-15, -5.838245629166617e-15, 95.34578677266671]])

In [15]:
job = pr.create.job.Lammps("equilibrate", delete_existing_job=True)
job.structure = structure
job.potential = 'FeH-NNIP'
job.calc_md(temperature=300, n_ionic_steps=1000)
job.server.cores = 16
job.run(delete_existing_job=True)
sample = job.get_structure(frame=-1)
job.animate_structure()

The job equilibrate was saved and received the ID: 10946


/tmp/ipykernel_78759/819370485.py:8: DeprecationWarning: pyiron_atomistics.atomistics.job.atomistic.animate_structure is deprecated: Call animate_structures() instead.  Arguments stride/center_of_mass/atom_indices/snapshot_indices/repeat can be emulated by calling trajectory() first..
  job.animate_structure()


NGLWidget(max_frame=10)

In [16]:
job = pr.create.job.Lammps("deform", delete_existing_job=True)
job.structure = sample.copy()
job.potential = 'FeH-NNIP'
job.calc_md(temperature=300, n_ionic_steps=3000000)
job.server.cores = 16
job.input.control["fix"] = "fix def1 all deform 1 z erate 1e-6"

In [17]:
job.input.control

,Parameter,Value,Comment
0,units,metal,
1,dimension,3,
2,boundary,p p p,
3,atom_style,atomic,
4,read_data,structure.inp,
5,include,potential.inp,
6,fix___ensemble,all nvt temp 300.0 300.0 0.1,
7,variable___dumptime,equal 100,
8,variable___thermotime,equal 100,
9,fix,fix def1 all deform 1 z erate 1e-6,


In [18]:
job.run(delete_existing_job=True)


The job deform was saved and received the ID: 10947


2024-06-28 18:27:01,562 - pyiron_log - WARNING - Job aborted
2024-06-28 18:27:01,563 - pyiron_log - WARNING - --------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: ias9122
--------------------------------------------------------------------------
LAMMPS (7 Feb 2024 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0 0 0) to (51.66 48.705515 95.345787)
  2 by 2 by 4 MPI processor grid
  reading atoms ...
  20736 atoms
  read_data CPU = 0.057 seconds
ERROR: Could not find fix group ID def1 (src/modify.cpp:836)
Last command: fix fix def1 all deform 1 z erate 1e-6
--------------------

RuntimeError: Job aborted

In [ ]:
job.animate_structures()

In [ ]:
stress = -job.output.pressures[:,2,2]
z = job.output.cells[:,2, 2]
eps = (z -z[0])/z[0]
plt.plot(eps, stress)
plt.ylabel("Stress (GPa)")
plt.xlabel("Strain")
plt.axvline(x=0.15, c="tab:orange")